In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import cv2
%matplotlib inline

# YOLOv3

In [2]:
from math import floor

path = "Models\\yolov3\\"

classes = None
with open(path+'coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]
    
yoloNet = cv2.dnn.readNet(path+'yolov3-spp.weights', path+'yolov3-spp.cfg')

In [3]:
def getBoxes(outs):
     # Get specs of the box where an object has been detected
        
    class_ids = [] # Indicates the type of object that has been found (0: person)
    confidences = [] # The score that each class has had
    boxes = [] # Boxes coordinates
    
    #create bounding box 

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.1:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                w = int(detection[2] * frameWidth)
                h = int(detection[3] * frameHeight)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h]) #(xUpLeft, yUpLeft, widthObject, heigthObject)
    return class_ids, confidences, boxes
    
def getPeople(indices, class_ids, confidences, boxes, showBox = True):
    # Plots rectangle of every person detected if showBox = True, and
    # returns the area of each person detected
    
    area = [] # Rectangle Area
    personBoxes = []

    # Check if is people detection
    for i in indices:
        i = i[0]
        box = boxes[i]
        if class_ids[i]==0:
            area.append(box[2]*box[3]) # width*height
            personBoxes.append(box)
            label = str(classes[class_ids[i]])
            if showBox:
                cv2.rectangle(frame, (round(box[0]),round(box[1])), (round(box[0]+box[2]),round(box[1]+box[3])), (0, 0, 255), 2)
                cv2.putText(frame, label, (round(box[0])-10,round(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    return area, personBoxes

In [4]:
name = "VideoSkier"

cap = cv2.VideoCapture("Dataset\\"+name+".mp4")

# Get fps from video
fps = cap.get(cv2.CAP_PROP_FPS)

if (cap.isOpened()== False):
    print("Error opening video stream or file")

ret, frame = cap.read()

# Get frame shape
frameWidth = frame.shape[1]
frameHeight = frame.shape[0]

# Confidence margin to cut biggest person detected
mConf = 0.35

fourcc = cv2.VideoWriter_fourcc(*'MP4V')

save = input("Save model? 0: no, Other: Name of file. ")

if save != "0":
    out = cv2.VideoWriter('Output\\'+'BoxSk'+save+'.mp4', fourcc, fps, (frameWidth, frameHeight))

while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:
        
        yoloNet.setInput(cv2.dnn.blobFromImage(frame, 0.00392, (416,416), (0,0,0), True, crop=False))

        layer_names = yoloNet.getLayerNames()
        output_layers = [layer_names[i[0] - 1] for i in yoloNet.getUnconnectedOutLayers()]
        outs = yoloNet.forward(output_layers)

        class_ids, confidences, boxes = getBoxes(outs)

        indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.1)
        
        area, personBoxes = getPeople(indices, class_ids, confidences, boxes, showBox = True)
        
        if save != "0": out.write(frame)
        
        if area != []:
            idx = area.index(max(area))
            box = personBoxes[idx]
            
            posBiggest = [max(floor(box[0]-(mConf*box[2])),0), min(floor((box[0]+box[2])+(mConf*box[2])), frameWidth-1),
                        max(floor(box[1]-(mConf*box[3])),0), min(floor((box[1]+box[3])+(mConf*box[3])), frameHeight-1)]

            cv2.imshow('Frame', frame[posBiggest[2]:posBiggest[3],posBiggest[0]:posBiggest[1]])
        else:
            cv2.imshow('Frame', frame)

        # Press Q on keyboard to  exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    else:
        break
        
cap.release()
if save != "0": out.release()
cv2.destroyAllWindows()

Save model? 0: no, Other: Name of file. 0
